In [101]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow import keras

In [253]:
# import temperature data
data = pd.read_csv('temp/temperature.csv').values
data = np.delete(data,0,1)
print(data)
n_steps = 4

[[27.37 23.29]
 [28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]
 [27.6  24.13]
 [28.3  23.85]
 [28.77 22.79]
 [29.03 24.22]
 [27.6  23.43]
 [28.07 24.24]
 [28.63 23.02]
 [27.9  19.6 ]
 [27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]
 [27.93 22.63]]


In [254]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    x, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [285]:

# convert into input/output
x, y = split_sequences(data, n_steps)


# print(data[20:23])
for i in range(len(x)):
	print(x[i], y[i])
n_features=x.shape[2]
# X.shape
data.shape
print(x.shape[2])

[[27.37 23.29]
 [28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]] [28.37 19.5 ]
[[28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]] [29.07 17.68]
[[28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]] [28.53 19.27]
[[28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]] [27.7  19.13]
[[28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]] [27.07 20.2 ]
[[29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]] [28.27 21.39]
[[28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]] [28.13 20.72]
[[27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]] [27.33 20.3 ]
[[27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]] [27.1  21.56]
[[28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]] [28.53 22.15]
[[28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]] [28.33 23.65]
[[27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]] [27.6  24.13]
[[27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]
 [27.6  24.13]] [28.3  23.85]
[[28.53 22.15]
 [28.33 23

In [289]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features) ))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [291]:
# fit model
model.fit(x,y,epochs=1000, verbose=0)
# demonstrate prediction

In [264]:
xinput=np.array([[27.37,23.29]
,[28.77,20.87]
,[28.73,22.41]
,[28.13,20.35]])
xinput = xinput.reshape((1, n_steps, n_features))
temp = model.predict(xinput, verbose= 0)
print(temp)


[[28.222849 20.298178]]


In [339]:
newModel = Sequential()
newModel.add(LSTM(100,activation='relu',input_shape=(1,4,2)))
newModel.add(LSTM(100, activation='relu',stateful=True))
newModel.add(Dense(2))
newModel.compile(optimizer='adam', loss='mse')
newModel.set_weights(model.get_weights())

In [340]:
newtemp = x[-1]
print(newtemp)
pp=np.delete(newtemp, obj=[0,0,1])
pp=pp.flatten()
pp=np.append(pp, values = temp.flatten())
print(pp)
pp.shape
pp=pp.reshape((1, n_steps, n_features))



[[27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]]
[27.07       20.8        28.47       22.6        28.17       22.63
 28.22284889 20.29817772]


In [341]:
predict_the_future = newModel.predict(pp, verbose=0)



In [342]:
print(predict_the_future)
for i in range(8):
    pp=np.delete(newtemp, obj=[0,0,1])
    pp=pp.flatten()
    pp=np.append(pp, values = predict_the_future.flatten())
    pp=pp.reshape((1, n_steps, n_features))
    print(pp)
    predict_the_future = newModel.predict(pp, verbose=0)
    

[[28.039291 20.616282]]
[[[27.07       20.8       ]
  [28.47       22.6       ]
  [28.17       22.63      ]
  [28.03929138 20.61628151]]]
[[[ 27.07        20.8       ]
  [ 28.47        22.6       ]
  [ 28.17        22.63      ]
  [207.61372375 203.34968567]]]
[[[  27.07         20.8       ]
  [  28.47         22.6       ]
  [  28.17         22.63      ]
  [1865.75695801 1003.55029297]]]
[[[   27.07          20.8       ]
  [   28.47          22.6       ]
  [   28.17          22.63      ]
  [12625.20605469  6760.25634766]]]
[[[2.70700000e+01 2.08000000e+01]
  [2.84700000e+01 2.26000000e+01]
  [2.81700000e+01 2.26300000e+01]
  [6.79926484e+04 3.78163438e+04]]]
[[[2.70700000e+01 2.08000000e+01]
  [2.84700000e+01 2.26000000e+01]
  [2.81700000e+01 2.26300000e+01]
  [3.63062750e+05 2.20029375e+05]]]
[[[2.70700000e+01 2.08000000e+01]
  [2.84700000e+01 2.26000000e+01]
  [2.81700000e+01 2.26300000e+01]
  [1.87971638e+06 1.20487125e+06]]]
[[[2.707000e+01 2.080000e+01]
  [2.847000e+01 2.260000e+01